# Import stuff

In [1]:
!pip install pprintpp

In [2]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32894 sha256=0b6efc1556b02bb9cfe92a7ae41d1fd803405291be3afbb4f254d44ff24c56d6
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [3]:
from tika import parser
import re
from google.colab import drive
drive.mount('/content/gdrive')
from collections import Counter
import spacy
from spacy import displacy
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
from collections import Counter
import en_core_web_sm
import pprintpp
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = en_core_web_sm.load()
import numpy as np

Mounted at /content/gdrive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Actors / Departments

In [4]:
#same approach as you had in the main class too, create a list with the files and then just chose the respective one
file_list = [
             '04_SOP-Business-Plan.pdf', #0
             'AccountsPayablePolicy.pdf', #1
             'purchasing-payment-policy.pdf', #2
             'UniversityofMichiganGift-Cards1.pdf' #3
]
file_index = 3
raw = "/content/gdrive/My Drive/Signavio_Test/data/"+file_list[file_index]

#convert pdf and call the dataframe own (I can't recall why I called it like this haha :D)
own = parser.from_file(raw)["content"]
own

2021-06-30 09:37:27,876 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-06-30 09:37:28,616 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-06-30 09:37:29,044 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSECTION:\n\n\n \n \n\nPage 1 of 4 \n\nSUBJECT:  Gift Cards – Process Documentation \nAPPLIES TO:  Purchasing, securing, and disbursing of gift cards  \n \nGift Card Overview \nThe objective of the gift card process is to ensure all gift cards are obtained, tracked, and distributed consistent with \nUniversity guidelines. \n \nNOTE: If using gift cards for human subjects, refer to the Human Subject Incentives written procedure template. \n \nGift Card Process \n\nActivity Responsibility Reference/Comments \nPURCHASING/APPROVING \n\nGift cards may be used for the following \nreason(s)/situation(s): \n \n• Employee recognition/appreciation \n\n  \n\nDetermine the number of gift cards needed for a \nspecific event. \n\nDepartment \nDesignated Staff  \n(See Appendix A) \n\nUnit should only purchase the \nexact number of gift cards that \nwill be used within 30 days. \n\nPurchase gift cards

In [5]:
#I have created a second df called owncleaned, which I will use to do some preprocessing on
#I did it in case I ruin it, so I can always go back to this cell and overwrite all the changes
owncleaned = own
owncleaned

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSECTION:\n\n\n \n \n\nPage 1 of 4 \n\nSUBJECT:  Gift Cards – Process Documentation \nAPPLIES TO:  Purchasing, securing, and disbursing of gift cards  \n \nGift Card Overview \nThe objective of the gift card process is to ensure all gift cards are obtained, tracked, and distributed consistent with \nUniversity guidelines. \n \nNOTE: If using gift cards for human subjects, refer to the Human Subject Incentives written procedure template. \n \nGift Card Process \n\nActivity Responsibility Reference/Comments \nPURCHASING/APPROVING \n\nGift cards may be used for the following \nreason(s)/situation(s): \n \n• Employee recognition/appreciation \n\n  \n\nDetermine the number of gift cards needed for a \nspecific event. \n\nDepartment \nDesignated Staff  \n(See Appendix A) \n\nUnit should only purchase the \nexact number of gift cards that \nwill be used within 30 days. \n\nPurchase gift cards

In [6]:
#replace all line breaks with ""
owncleaned = owncleaned.replace("\n","")
#do some text preprocessing
owncleaned = [strip_numeric(data) for data in owncleaned]
owncleaned = [strip_punctuation(data) for data in owncleaned]
owncleaned = [strip_multiple_whitespaces(data) for data in owncleaned]
owncleaned = ("").join(owncleaned)
#remove stopwords
nltk_stopwords = set(stopwords.words('english'))
owncleaned = remove_stopwords(owncleaned)
#show the preprocessed text
owncleaned

'SECTION Page SUBJECT Gift Cards – Process Documentation APPLIES TO Purchasing securing disbursing gift cards Gift Card Overview The objective gift card process ensure gift cards obtained tracked distributed consistent University guidelines NOTE If gift cards human subjects refer Human Subject Incentives written procedure template Gift Card Process Activity Responsibility Reference Comments PURCHASING APPROVING Gift cards following reason s situation s • Employee recognition appreciation Determine number gift cards needed specific event Department Designated Staff See Appendix A Unit purchase exact number gift cards days Purchase gift cards P card reimbursed Concur Department Designated Staff Appendix A Review gift card purchase ensure appropriate valid Department Higher Administrative Authority See Appendix A Individuals approving requests purchases gift cards DISTRIBUTING TRACKING Individuals handling distributing gift cards complete cash handling training course offered Treasury Dep

In [7]:
#run spacy name entity recognition model on it and store results of organization in mostcommon
doc = nlp(owncleaned)
items = [x.text for x in doc.ents if x.label_ in ["ORG"]]
mostcommon = np.array(Counter(items).most_common())

#return all entities, which appear more often than 2 times
for i, j in mostcommon:
  if int(j) >= 3:
    print(i,j)

Department Higher Administrative Authority 9
PeoplePay 7
FBS 3


In [8]:
#You can leave this out!
#this is actually only for our information and doesn't add any value to the above code. Just intersting to see how many org. and so on were found
#check all labels and the respective counts of the labels
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'DATE': 4,
         'EVENT': 1,
         'GPE': 1,
         'LAW': 1,
         'ORG': 52,
         'PERSON': 4,
         'WORK_OF_ART': 3})